In [1]:
import gym
import scipy
import numpy as np

In [2]:
# 导入‘CliffWalking-v0’环境

env = gym.make('CliffWalking-v0')
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测数量 = {}, 动作数量 = {}'.format(env.nS, env.nA))
print('地图大小 = {}'.format(env.shape))

观测空间 = Discrete(48)
动作空间 = Discrete(4)
观测数量 = 48, 动作数量 = 4
地图大小 = (4, 12)


In [3]:
# 运行一个回合

def play_once(env, policy):
    total_reward = 0
    state = env.reset()
    
    while True:
        loc = np.unravel_index(state, env.shape)
        print('状态 = {}, 位置 = {}'.format(state, loc), end = ' | ')
        action = np.random.choice(env.nA, p=policy[state])
        state, reward, done, _ = env.step(action)
        print('动作 = {}, 奖励 = {}'.format(action, reward))
        total_reward += reward
        if done:
            break
    
    return total_reward

In [4]:
# 最优策略

actions = np.ones(env.shape, dtype=int)
actions[-1, :] = 0
actions[:, -1] = 2
optimal_policy = np.eye(4)[actions.reshape(-1)]
print('最优策略 = \n{}\n'.format(actions))

total_reward = play_once(env, optimal_policy)
print('总奖励 = {}'.format(total_reward))

最优策略 = 
[[1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 0 2]]

状态 = 36, 位置 = (3, 0) | 动作 = 0, 奖励 = -1
状态 = 24, 位置 = (2, 0) | 动作 = 1, 奖励 = -1
状态 = 25, 位置 = (2, 1) | 动作 = 1, 奖励 = -1
状态 = 26, 位置 = (2, 2) | 动作 = 1, 奖励 = -1
状态 = 27, 位置 = (2, 3) | 动作 = 1, 奖励 = -1
状态 = 28, 位置 = (2, 4) | 动作 = 1, 奖励 = -1
状态 = 29, 位置 = (2, 5) | 动作 = 1, 奖励 = -1
状态 = 30, 位置 = (2, 6) | 动作 = 1, 奖励 = -1
状态 = 31, 位置 = (2, 7) | 动作 = 1, 奖励 = -1
状态 = 32, 位置 = (2, 8) | 动作 = 1, 奖励 = -1
状态 = 33, 位置 = (2, 9) | 动作 = 1, 奖励 = -1
状态 = 34, 位置 = (2, 10) | 动作 = 1, 奖励 = -1
状态 = 35, 位置 = (2, 11) | 动作 = 2, 奖励 = -1
总奖励 = -13


In [5]:
# 用Bellman方程求解状态价值和动作价值

def evaluate_bellman(env, policy, gamma=1.0):
    a, b = np.eye(env.nS), np.zeros((env.nS))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            pi = policy[state][action]
            for p, next_state, reward, done in env.P[state][action]:
                a[state, next_state] -= pi * gamma * p
                b[state] += pi * reward * p
    v = np.linalg.solve(a, b)
    
    q = np.zeros((env.nS, env.nA))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            for p, next_state, reward, done in env.P[state][action]:
                q[state][action] += (reward + gamma * v[next_state]) * p
    return v, q

In [6]:
# 评估随机策略

policy = np.random.uniform(size=(env.nS, env.nA))
policy = policy / np.sum(policy, axis=1)[:, np.newaxis]
state_values, action_values = evaluate_bellman(env, policy)
print('状态价值 = {}'.format(state_values))
print('动作价值 = {}'.format(action_values))

状态价值 = [-84537767.17599674 -84537734.14924777 -84537128.96237774
 -84536314.46853566 -84534411.5386176  -84523673.65553102
 -84484789.05526869 -84373265.75787517 -84329922.07866065
 -84275296.29400891 -84157473.9489998  -84126181.5476279
 -84537790.11340794 -84537698.61188367 -84537527.58746669
 -84537068.71889049 -84535609.97717507 -84519784.83687702
 -84501674.3798621  -84427458.1592079  -84303436.46428464
 -84229578.6346756  -84101343.11962558 -84084223.0419421
 -84538076.4686468  -84538041.2098556  -84537974.47486386
 -84537748.23394884 -84536900.56879608 -84526997.16611306
 -84524811.57365067 -84514151.94362962 -84382801.25759234
 -84220081.03580694 -83779780.71472944 -82915912.33177578
 -84538305.52849488 -84538312.12583822 -84538153.59525718
 -84538145.98539561 -84538403.63886732 -84535498.19949386
 -84533771.18294747 -84535051.68012775 -84496447.7055068
 -84499725.97483775 -63199813.39233176         0.        ]
动作价值 = [[-8.45377682e+07 -8.45377351e+07 -8.45377911e+07 -8.4537768

In [7]:
# 评估最优策略

optimal_state_values, optimal_action_values = evaluate_bellman(env, optimal_policy)
print('最优状态价值 = {}'.format(optimal_state_values))
print('最优动作价值 = {}'.format(optimal_action_values))

最优状态价值 = [-14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3. -13. -12.
 -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2. -12. -11. -10.  -9.
  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1. -13. -12. -11. -10.  -9.  -8.
  -7.  -6.  -5.  -4.  -3.   0.]
最优动作价值 = [[ -15.  -14.  -14.  -15.]
 [ -14.  -13.  -13.  -15.]
 [ -13.  -12.  -12.  -14.]
 [ -12.  -11.  -11.  -13.]
 [ -11.  -10.  -10.  -12.]
 [ -10.   -9.   -9.  -11.]
 [  -9.   -8.   -8.  -10.]
 [  -8.   -7.   -7.   -9.]
 [  -7.   -6.   -6.   -8.]
 [  -6.   -5.   -5.   -7.]
 [  -5.   -4.   -4.   -6.]
 [  -4.   -4.   -3.   -5.]
 [ -15.  -13.  -13.  -14.]
 [ -14.  -12.  -12.  -14.]
 [ -13.  -11.  -11.  -13.]
 [ -12.  -10.  -10.  -12.]
 [ -11.   -9.   -9.  -11.]
 [ -10.   -8.   -8.  -10.]
 [  -9.   -7.   -7.   -9.]
 [  -8.   -6.   -6.   -8.]
 [  -7.   -5.   -5.   -7.]
 [  -6.   -4.   -4.   -6.]
 [  -5.   -3.   -3.   -5.]
 [  -4.   -3.   -2.   -4.]
 [ -14.  -12.  -14.  -13.]
 [ -13.  -11. -113.  -13.]
 [ -12.  -10. -113.  -12.]
 [ -11. 

In [8]:
# 用线性规划求解Bellman最优方程

def optimal_bellman(env, gamma=1.0):
    p = np.zeros((env.nS, env.nA, env.nS))
    r = np.zeros((env.nS, env.nA))
    for state in range(env.nS - 1):
        for action in range(env.nA):
            for prob, next_state, reward, done in env.P[state][action]:
                p[state, action, next_state] += prob
                r[state, action] += reward * prob
    
    c = np.ones((env.nS))
    a_ub = gamma * p.reshape(-1, env.nS) - np.repeat(np.eye(env.nS), env.nA, axis=0)
    b_ub = -r.reshape(-1)
    
    bounds = [(None, None),] * env.nS
    res = scipy.optimize.linprog(c, a_ub, b_ub, bounds=bounds, method='interior-point')
    v = res.x
    q = r + gamma * np.dot(p, v)
    return v, q

optimal_state_values, optimal_action_values = optimal_bellman(env)
print('最优状态价值 = {}'.format(optimal_state_values))
print('最优动作价值 = {}'.format(optimal_action_values))

最优状态价值 = [-1.40000000e+01 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01
 -1.00000000e+01 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00
 -6.00000000e+00 -5.00000000e+00 -4.00000000e+00 -3.00000000e+00
 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01
 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00
 -5.00000000e+00 -4.00000000e+00 -3.00000000e+00 -2.00000000e+00
 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01 -9.00000000e+00
 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00 -5.00000000e+00
 -4.00000000e+00 -3.00000000e+00 -2.00000000e+00 -1.00000000e+00
 -1.30000000e+01 -1.20000000e+01 -1.10000000e+01 -1.00000000e+01
 -9.00000000e+00 -8.00000000e+00 -7.00000000e+00 -6.00000000e+00
 -5.00000000e+00 -4.00000000e+00 -9.99999999e-01  1.82270928e-11]
最优动作价值 = [[ -14.99999999  -13.99999999  -13.99999999  -14.99999999]
 [ -13.99999999  -13.          -13.          -14.99999999]
 [ -13.          -12.          -12.          -13.99999999]
 [ -12.          -11.   

In [9]:
# 用最优动作价值确定最优确定性策略

optimal_actions = optimal_action_values.argmax(axis=1)
print('最优策略 = \n{}'.format(optimal_actions.reshape(env.shape)))

最优策略 = 
[[2 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1 1 1 1 2]
 [0 0 0 0 0 0 0 0 0 0 1 0]]
